In [ ]:
import torch
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import torch.nn.functional as F

from torch import nn
from torch.utils.data import DataLoader, RandomSampler
from torchvision import datasets, transforms

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.metrics.confusion_matrix import ConfusionMatrix
from ignite.handlers import EarlyStopping

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"        # bez tego wysadza kernel kiedy rysuje obrazek
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"  # potrzebne dla deterministycznego działania

In [ ]:
classes = [
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device used is {device}")


torch.use_deterministic_algorithms(True)
torch.manual_seed(0)

BATCH_SIZE = 64
COMMITTEE_SIZE = 5

In [ ]:
def get_loaders_single_model(batch_size):
    train_data = datasets.CIFAR10("./cifar10", download=True, transform=transforms.ToTensor(), train=True)
    test_data = datasets.CIFAR10("./cifar10", download=True, transform=transforms.ToTensor(), train=False)

    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

    return train_loader, test_loader


def get_loaders_committee(batch_size, committee_size):
    train_data = datasets.CIFAR10("./cifar10", download=True, transform=transforms.ToTensor(), train=True)
    test_data = datasets.CIFAR10("./cifar10", download=True, transform=transforms.ToTensor(), train=False)

    train_loaders = [DataLoader(train_data, batch_size=batch_size, sampler=RandomSampler(train_data, True, len(train_data))) for _ in range(committee_size)]
    test_loader = DataLoader(test_data, batch_size=batch_size)

    return train_loaders, test_loader

In [ ]:
# for single model
train_loader, test_loader = get_loaders_single_model(BATCH_SIZE)

In [ ]:
# for committee
train_loaders, test_loader = get_loaders_committee(BATCH_SIZE, COMMITTEE_SIZE)

In [ ]:
def print_img(img, axs):
    img_np = img.numpy()
    img_denormalized = (img_np*255).astype("uint8").transpose(1, 2, 0) 
    return axs.imshow(img_denormalized)

In [ ]:
fig, axs = plt.subplots(3, 5, constrained_layout=True)
axs = np.reshape(axs, -1)
for x in range(15):
    img, label = train_data[x]
    axs[x].title.set_text(classes[label])
    print_img(img, axs[x])

In [ ]:
from typing import Any, Callable, cast, Tuple, Union

import torch

from ignite import distributed as idist
from ignite.exceptions import NotComputableError
from ignite.metrics import EpochMetric


def roc_auc_compute_fn(y_preds: torch.Tensor, y_targets: torch.Tensor) -> float:
    from sklearn.metrics import roc_auc_score

    y_true = y_targets.cpu().numpy()
    y_pred = y_preds.cpu().numpy()
    return roc_auc_score(y_true, y_pred, multi_class='ovr')


def roc_auc_curve_compute_fn(y_preds: torch.Tensor, y_targets: torch.Tensor) -> Tuple[Any, Any, Any]:
    from sklearn.metrics import roc_curve

    y_true = y_targets.cpu().numpy()
    y_pred = y_preds.cpu().numpy()
    return roc_curve(y_true, y_pred)



class AUC(EpochMetric):
    """Computes Area Under the Receiver Operating Characteristic Curve (ROC AUC)
    accumulating predictions and the ground-truth during an epoch and applying
    `sklearn.metrics.roc_auc_score <https://scikit-learn.org/stable/modules/generated/
    sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score>`_ .

    Args:
        output_transform: a callable that is used to transform the
            :class:`~ignite.engine.engine.Engine`'s ``process_function``'s output into the
            form expected by the metric. This can be useful if, for example, you have a multi-output model and
            you want to compute the metric with respect to one of the outputs.
        check_compute_fn: Default False. If True, `roc_curve
            <https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#
            sklearn.metrics.roc_auc_score>`_ is run on the first batch of data to ensure there are
            no issues. User will be warned in case there are any issues computing the function.
        device: optional device specification for internal storage.

    Note:

        ROC_AUC expects y to be comprised of 0's and 1's. y_pred must either be probability estimates or confidence
        values. To apply an activation to y_pred, use output_transform as shown below:

        .. code-block:: python

            def sigmoid_output_transform(output):
                y_pred, y = output
                y_pred = torch.sigmoid(y_pred)
                return y_pred, y
            avg_precision = ROC_AUC(sigmoid_output_transform)

    Examples:

        .. include:: defaults.rst
            :start-after: :orphan:

        .. testcode::

            roc_auc = ROC_AUC()
            #The ``output_transform`` arg of the metric can be used to perform a sigmoid on the ``y_pred``.
            roc_auc.attach(default_evaluator, 'roc_auc')
            y_pred = torch.tensor([[0.0474], [0.5987], [0.7109], [0.9997]])
            y_true = torch.tensor([[0], [0], [1], [0]])
            state = default_evaluator.run([[y_pred, y_true]])
            print(state.metrics['roc_auc'])

        .. testoutput::

            0.6666...
    """

    def __init__(
        self,
        output_transform: Callable = lambda x: x,
        check_compute_fn: bool = False,
        device: Union[str, torch.device] = torch.device("cpu"),
    ):

        try:
            from sklearn.metrics import roc_auc_score  # noqa: F401
        except ImportError:
            raise ModuleNotFoundError("This contrib module requires scikit-learn to be installed.")

        super(AUC, self).__init__(
            roc_auc_compute_fn, output_transform=output_transform, check_compute_fn=check_compute_fn, device=device
        )




class RocCurve(EpochMetric):
    """Compute Receiver operating characteristic (ROC) for binary classification task
    by accumulating predictions and the ground-truth during an epoch and applying
    `sklearn.metrics.roc_curve <https://scikit-learn.org/stable/modules/generated/
    sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve>`_ .

    Args:
        output_transform: a callable that is used to transform the
            :class:`~ignite.engine.engine.Engine`'s ``process_function``'s output into the
            form expected by the metric. This can be useful if, for example, you have a multi-output model and
            you want to compute the metric with respect to one of the outputs.
        check_compute_fn: Default False. If True, `sklearn.metrics.roc_curve
            <https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#
            sklearn.metrics.roc_curve>`_ is run on the first batch of data to ensure there are
            no issues. User will be warned in case there are any issues computing the function.
        device: optional device specification for internal storage.

    Note:
        RocCurve expects y to be comprised of 0's and 1's. y_pred must either be probability estimates or confidence
        values. To apply an activation to y_pred, use output_transform as shown below:

        .. code-block:: python

            def sigmoid_output_transform(output):
                y_pred, y = output
                y_pred = torch.sigmoid(y_pred)
                return y_pred, y
            avg_precision = RocCurve(sigmoid_output_transform)

    Examples:

        .. include:: defaults.rst
            :start-after: :orphan:

        .. testcode::

            roc_auc = RocCurve()
            #The ``output_transform`` arg of the metric can be used to perform a sigmoid on the ``y_pred``.
            roc_auc.attach(default_evaluator, 'roc_auc')
            y_pred = torch.tensor([0.0474, 0.5987, 0.7109, 0.9997])
            y_true = torch.tensor([0, 0, 1, 0])
            state = default_evaluator.run([[y_pred, y_true]])
            print("FPR", [round(i, 3) for i in state.metrics['roc_auc'][0].tolist()])
            print("TPR", [round(i, 3) for i in state.metrics['roc_auc'][1].tolist()])
            print("Thresholds", [round(i, 3) for i in state.metrics['roc_auc'][2].tolist()])

        .. testoutput::

            FPR [0.0, 0.333, 0.333, 1.0]
            TPR [0.0, 0.0, 1.0, 1.0]
            Thresholds [2.0, 1.0, 0.711, 0.047]

    ..  versionchanged:: 0.4.11
        added `device` argument
    """

    def __init__(
        self,
        output_transform: Callable = lambda x: x,
        check_compute_fn: bool = False,
        device: Union[str, torch.device] = torch.device("cpu"),
    ) -> None:

        try:
            from sklearn.metrics import roc_curve  # noqa: F401
        except ImportError:
            raise ModuleNotFoundError("This contrib module requires scikit-learn to be installed.")

        super(RocCurve, self).__init__(
            roc_auc_curve_compute_fn,  # type: ignore[arg-type]
            output_transform=output_transform,
            check_compute_fn=check_compute_fn,
            device=device,
        )


    def compute(self) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:  # type: ignore[override]
        if len(self._predictions) < 1 or len(self._targets) < 1:
            raise NotComputableError("RocCurve must have at least one example before it can be computed.")

        _prediction_tensor = torch.cat(self._predictions, dim=0)
        _target_tensor = torch.cat(self._targets, dim=0)

        ws = idist.get_world_size()
        if ws > 1:
            # All gather across all processes
            _prediction_tensor = cast(torch.Tensor, idist.all_gather(_prediction_tensor))
            _target_tensor = cast(torch.Tensor, idist.all_gather(_target_tensor))

        if idist.get_rank() == 0:
            # Run compute_fn on zero rank only
            fpr, tpr, thresholds = cast(Tuple, self.compute_fn(_prediction_tensor, _target_tensor))
            fpr = torch.tensor(fpr, device=_prediction_tensor.device)
            tpr = torch.tensor(tpr, device=_prediction_tensor.device)
            thresholds = torch.tensor(thresholds, device=_prediction_tensor.device)
        else:
            fpr, tpr, thresholds = None, None, None

        if ws > 1:
            # broadcast result to all processes
            fpr = idist.broadcast(fpr, src=0, safe_mode=True)
            tpr = idist.broadcast(tpr, src=0, safe_mode=True)
            thresholds = idist.broadcast(thresholds, src=0, safe_mode=True)

        return fpr, tpr, thresholds


In [ ]:
class Committee(nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = models

    def forward(self, x):
        predictions = torch.stack([model.forward(x).argmax(1) for model in self.models])
        predictions = torch.stack([predictions[:, i].bincount(minlength=10) for i in range(x.shape[0])]).float()
        return predictions


def plot_confusion_matrix(cm, draw=True, save=False, savefile=""):
    plt.figure()
    ax = plt.subplot()
    sns.heatmap(cm, annot=True, fmt='g', ax=ax)

    # labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix')
    ax.xaxis.set_ticklabels(classes)
    ax.yaxis.set_ticklabels(classes)

    if draw:
        plt.show()
    if save and savefile and savefile.strip():
        plt.savefig(f"plots/{savefile}.png")

def softmax_output_transform(output):
    y_pred, y = output
    y_pred = torch.softmax(y_pred, 1)
    return y_pred, y

def to_cpy_output_transform(output):
    y_pred, y = output
    return y_pred.to("cpu"), y.to("cpu")


def run_model(model, train_loader, test_loader, device=device, draw=True, save=False, savefile=""):
    model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    trainer = create_supervised_trainer(model, optimizer, loss_fn, device)

    val_metrics = {
        "accuracy": Accuracy(device=device),
        "loss": Loss(loss_fn, device=device),
        "auc": AUC(softmax_output_transform, device=device),
        "confusion_matrix": ConfusionMatrix(10, output_transform=to_cpy_output_transform)
    }

    train_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)
    val_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)


    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(trainer):
        train_evaluator.run(train_loader)
        metrics = train_evaluator.state.metrics
        print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy'] * 100:.2f}%, Avg loss: {metrics['loss']:.2f}, AUC: {metrics['auc']:.2f}")

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(trainer):
        val_evaluator.run(test_loader)
        metrics = val_evaluator.state.metrics
        # print(metrics["confusion_matrix"])
        print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy'] * 100:.2f}%, Avg loss: {metrics['loss']:.2f}, AUC: {metrics['auc']:.2f}")

    def score_function(engine):
        metrics = engine.state.metrics
        return metrics["accuracy"]
    

    val_evaluator.add_event_handler(Events.COMPLETED, EarlyStopping(3, score_function, trainer))

    trainer.run(train_loader, max_epochs=100)
    plot_confusion_matrix(val_evaluator.state.metrics["confusion_matrix"], draw, save, savefile)


def run_models(models, train_loaders, test_loader, device=device, draw=True, save=False, savefile=""):
    assert len(models) == len(train_loaders), "Number of models and number of train_loaders should be equal"

    for i, model in enumerate(models):
        print(f"Training model #{i + 1}...")

        model.to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

        trainer = create_supervised_trainer(model, optimizer, loss_fn, device)

        val_metrics = {
            "accuracy": Accuracy(device=device),
            "loss": Loss(loss_fn, device=device),
            "auc": AUC(softmax_output_transform, device=device)
        }

        train_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)
        val_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)


        @trainer.on(Events.EPOCH_COMPLETED)
        def log_training_results(trainer):
            train_evaluator.run(train_loaders[i])
            metrics = train_evaluator.state.metrics
            print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy'] * 100:.2f}%, Avg loss: {metrics['loss']:.2f}, AUC: {metrics['auc']:.2f}")

        @trainer.on(Events.EPOCH_COMPLETED)
        def log_validation_results(trainer):
            val_evaluator.run(test_loader)
            metrics = val_evaluator.state.metrics
            print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy'] * 100:.2f}%, Avg loss: {metrics['loss']:.2f}, AUC: {metrics['auc']:.2f}")

        def score_function(engine):
            metrics = engine.state.metrics
            return metrics["accuracy"]


        val_evaluator.add_event_handler(Events.COMPLETED, EarlyStopping(3, score_function, trainer))
        trainer.run(train_loaders[i], max_epochs=100)


    committee = Committee(models)
    
    loss_fn = nn.CrossEntropyLoss()
    val_metrics = {
        "accuracy": Accuracy(device=device),
        "loss": Loss(loss_fn, device=device),
        "auc": AUC(softmax_output_transform, device=device),
        "confusion_matrix": ConfusionMatrix(10, device=device)
    }
    evaluator = create_supervised_evaluator(committee, metrics=val_metrics, device=device)

    @evaluator.on(Events.COMPLETED)
    def log_committee_results(evaluator):
        metrics = evaluator.state.metrics
        print(f"Validation Results - Committee Avg accuracy: {metrics['accuracy'] * 100:.2f}%, Avg loss: {metrics['loss']:.2f}, AUC: {metrics['auc']:.2f}")

    evaluator.run(test_loader)
    plot_confusion_matrix(evaluator.state.metrics["confusion_matrix"], draw, save, savefile)

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self, conv_channels, kernel_sizes, fc_sizes):
        super().__init__()

        self.convs = nn.ModuleList()
        for i in range(len(kernel_sizes)):
            self.convs.append(nn.Conv2d(conv_channels[i], conv_channels[i + 1], kernel_sizes[i]))

        self.fcs = nn.ModuleList()
        for i in range(len(fc_sizes) - 1):
            self.fcs.append(nn.Linear(fc_sizes[i], fc_sizes[i + 1]))

        
    def forward(self, x):
        for conv in self.convs:
            x = F.relu(conv(x))
            
        x = torch.flatten(x, 1)
        for fc in self.fcs[:-1]:
            x = F.relu(fc(x))

        x = self.fcs[-1](x)
        return x


class PoolingNet(nn.Module):
    def __init__(self, conv_channels, kernel_sizes, pools, fc_sizes):
        super().__init__()

        self.convs = nn.ModuleList()
        for i in range(len(kernel_sizes)):
            if pools[i]:
                self.convs.append(nn.Sequential([
                    nn.Conv2d(conv_channels[i], conv_channels[i + 1], kernel_sizes[i]),
                    nn.MaxPool2d(2, 2)
                ]))
            else:
                self.convs.append(nn.Conv2d(conv_channels[i], conv_channels[i + 1], kernel_sizes[i]))

        self.fcs = nn.ModuleList()
        for i in range(len(fc_sizes) - 1):
            self.fcs.append(nn.Linear(fc_sizes[i], fc_sizes[i + 1]))

    def forward(self, x):
        for conv in self.convs:
            x = F.relu(conv(x))

        x = torch.flatten(x, 1)
        for fc in self.fcs[:-1]:
            x = F.relu(fc(x))

        x = self.fcs[-1](x)
        return x


class ResidualNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = ResidualNet._conv_block(3, 64, True)
        self.conv2 = ResidualNet._conv_block(64, 128, True)
        self.conv3 = ResidualNet._conv_block(128, 128)

        self.fc1 = nn.Linear(512 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(self.conv3(x)) + x

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    
    @staticmethod
    def _conv_block(in_channels, out_channels, pool=False):
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
                nn.BatchNorm2d(out_channels), 
                nn.ReLU(inplace=True)]
        if pool:
            layers.append(nn.MaxPool2d(2))
        return nn.Sequential(*layers)

In [ ]:
model = SimpleNet([3, 6, 16, 32, 64], [5, 5, 5, 5], [64 * 16 * 16, 120, 84, 32, 10])
run_model(model, train_loader, test_loader, draw=False, save=True, savefile="SimpleNet_kernel_5_Single")

model = SimpleNet([3, 6, 16, 32, 64], [3, 3, 3, 3], [64 * 16 * 16, 120, 84, 32, 10])
run_model(model, train_loader, test_loader, draw=False, save=True, savefile="SimpleNet_kernel_3_Single")

model = PoolingNet([3, 6, 16, 32], [3, 3, 3, 3], [True, True, False], [32 * 4 * 4, 120, 84, 64, 32, 10])
run_model(model, train_loader, test_loader, draw=False, save=True, savefile="PoolingNet_Single")

model = ResidualNet()
run_model(model, train_loader, test_loader, draw=False, save=True, savefile="ResidualNet_Single")


train_loaders, test_loader = get_loaders_committee(BATCH_SIZE, COMMITTEE_SIZE)

models = [SimpleNet([3, 6, 16, 32, 64], [5, 5, 5, 5], [64 * 16 * 16, 120, 84, 32, 10]) for _ in range(COMMITTEE_SIZE)]
run_models(models, train_loaders, test_loader, draw=False, save=True, savefile="SimpleNet_kernel_5_Committee")

models = [SimpleNet([3, 6, 16, 32, 64], [3, 3, 3, 3], [64 * 16 * 16, 120, 84, 32, 10]) for _ in range(COMMITTEE_SIZE)]
run_models(models, train_loaders, test_loader, draw=False, save=True, savefile="SimpleNet_kernel_3_Committee")

models = [PoolingNet([3, 6, 16, 32], [3, 3, 3, 3], [True, True, False], [32 * 4 * 4, 120, 84, 64, 32, 10]) for _ in range(COMMITTEE_SIZE)]
run_models(models, train_loaders, test_loader, draw=False, save=True, savefile="PoolingNet_Committee")

model = [ResidualNet() for _ in range(COMMITTEE_SIZE)]
run_model(model, train_loader, test_loader, draw=False, save=True, savefile="ResidualNet_Single")

1 + 3 - Validation Results - Epoch[11] Avg accuracy: 55.59%, Avg loss: 1.34
2 + 3 - Validation Results - Epoch[10] Avg accuracy: 58.85%, Avg loss: 1.34

3 + 3 - Validation Results - Epoch[10] Avg accuracy: 58.77%, Avg loss: 1.23
3 + 4 - Validation Results - Epoch[12] Avg accuracy: 61.44%, Avg loss: 1.42
3 + 5 - Validation Results - Epoch[15] Avg accuracy: 57.74%, Avg loss: 1.47

4 + 3 - Validation Results - Epoch[11] Avg accuracy: 59.61%, Avg loss: 1.36
4 + 4 - Validation Results - Epoch[15] Avg accuracy: 61.82%, Avg loss: 1.52
4 + 5 - Validation Results - Epoch[19] Avg accuracy: 60.91%, Avg loss: 1.24

5 + 3 - Validation Results - Epoch[18] Avg accuracy: 60.97%, Avg loss: 1.17
5 + 4 - Validation Results - Epoch[14] Avg accuracy: 60.22%, Avg loss: 1.24
5 + 5 - Validation Results - Epoch[23] Avg accuracy: 61.34%, Avg loss: 1.33
5 + 6 - Validation Results - Epoch[20] Avg accuracy: 59.16%, Avg loss: 1.23

6 + 3 - Validation Results - Epoch[17] Avg accuracy: 60.48%, Avg loss: 1.22


# with pooling
2 + 2 - Validation Results - Epoch[16] Avg accuracy: 61.48%, Avg loss: 1.09
2 + 3 - Validation Results - Epoch[19] Avg accuracy: 61.46%, Avg loss: 1.11
2 + 4 - Validation Results - Epoch[17] Avg accuracy: 61.60%, Avg loss: 1.10
2 + 5 - Validation Results - Epoch[17] Avg accuracy: 58.76%, Avg loss: 1.23

2 + 2 - Validation Results - Epoch[25] Avg accuracy: 63.66%, Avg loss: 1.10  # 3x3 kernel

2 + 2 - Validation Results - Epoch[65] Avg accuracy: 59.91%, Avg loss: 1.17  # sigmoid
2 + 3 - Validation Results - Epoch[31] Avg accuracy: 56.99%, Avg loss: 1.21  # sigmoid
2 + 4 - Validation Results - Epoch[40] Avg accuracy: 59.25%, Avg loss: 1.17  # sigmoid
2 + 2 - Validation Results - Epoch[22] Avg accuracy: 52.06%, Avg loss: 1.33  # 3x3 kernel sigmoid

3 + 4 - Validation Results - Epoch[15] Avg accuracy: 65.09%, Avg loss: 1.09  # 3x3 kernel, no pooling on last conv, no relu between conv and pool
3 + 5 - Validation Results - Epoch[20] Avg accuracy: 65.20%, Avg loss: 1.11  # 3x3 kernel, no pooling on last conv, no relu between conv and pool
3 + 5 - Validation Results - Epoch[22] Avg accuracy: 64.24%, Avg loss: 1.11  # 3v3 kernel, no pooling on last conv

3 + 5 - Validation Results - Epoch[14] Avg accuracy: 71.76%, Avg loss: 0.91  # 3v3 kernel, no pooling on last conv, 3 -> 64 -> 128 -> 128 channels

Validation Results - Epoch[11] Avg accuracy: 76.68%, Avg loss: 0.89  # architecture from https://medium.com/analytics-vidhya/resnet-10f4ef1b9d4c + 5 fc
Validation Results - Epoch[14] Avg accuracy: 77.51%, Avg loss: 0.99  # j.w. + 4 fc, last with relu
Validation Results - Epoch[11] Avg accuracy: 77.01%, Avg loss: 0.81  # architecture from https://medium.com/analytics-vidhya/resnet-10f4ef1b9d4c -- 3 conv + 5 fc, res on last conv
Validation Results - Epoch[16] Avg accuracy: 77.53%, Avg loss: 0.96  # architecture from https://medium.com/analytics-vidhya/resnet-10f4ef1b9d4c -- 2 conv + res + 4 fc, res on last conv which is double (actually as in article)  -- left in model in commit
Validation Results - Epoch[13] Avg accuracy: 67.98%, Avg loss: 1.30  # same as above but two res modules
Validation Results - Epoch[12] Avg accuracy: 70.42%, Avg loss: 1.37  # 2 conv + res + 1 conv + 4 fc
Validation Results - Epoch[10] Avg accuracy: 68.59%, Avg loss: 1.47  # 2 conv + res + 2 conv + res + 4 fc
Validation Results - Epoch[11] Avg accuracy: 69.94%, Avg loss: 1.33  # 2 conv + res + 2 conv + res + 5 fc

Validation Results - Committee Avg accuracy: 81.64%, Avg loss: 0.65, AUC: 0.97  # best residual + committee